# LDA Spark Framework
<p class="lead">This <a href="http://localhost:8889/notebooks/Project/IdeaBox/Spark%20Framework.ipynb">notebook</a>
defines the framework to train the LDA model and classify to the respective categories. </p>

<div class="alert alert-warning">
    <strong>Version:</strong> 1.0.1 <br>
    <strong>Last-Modified:</strong>	2017-03-10 (Fri, 10 March 2017) <br>
    <strong>Author:</strong>	Ben <br>
    <strong>Status:</strong>	Initial <br>
    <strong>Created:</strong> 2017-03-08 <br>
</div>

In [ ]:
import findspark
findspark.init()

In [1]:
import math
import nltk   
import pyspark
import re

from bs4 import BeautifulSoup
from collections import Counter, defaultdict
from pyspark import SparkContext
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.mllib.clustering import LDA, LDAModel
from random import random
from urllib import urlopen

<div class="lead" style="font-size:110%">
    <strong>NOTE (for Alipay):</strong><br>
    <strong>num_of_stop_words</strong> = 10<br> 
    # Number of most common words to remove, trying to eliminate stop words<br>
    <strong>num_topics</strong> = 1<br>
    # Number of topics we are looking for<br>
    <strong>num_words_per_topic</strong> = 100<br>
    # Number of words to display for each topic<br>
    <strong>max_iterations</strong> = 50<br>
    # Max number of times to iterate before finishing<br>
    <br>
    TODO: add Word2Vec for the cosine similarity
</div>

In [2]:
num_of_stop_words = 10 
num_topics = 1
num_words_per_topic = 100
max_iterations = 50   

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Adding a series of keywords for each of the corresponding categories defined for Alipay
</div>

In [3]:
topic_list = []

ZH_CATS = {}
ZH_CATS["CASINO"] = [u'赌博', u'博彩', u'筹码', u'赌城', u'轮盘', u'罗盘度', u'俄罗斯轮盘', u'销金窟',                        
                     u'牌九', u'赌局', u'押宝', u'六合彩', u'彩票', u'斗牌', u'掷色子', u'赛马', u'足智彩', u'赛狗', 
                     u'网上百家乐', u'扎金花', u'拉耗子', u'捕鱼机', u'21点', u'老虎机', u'网络博彩', u'赌球', 
                     u'赌马', u'投注', u'娱乐城', u'网络庄闲和', u'庄家', u'外围', u'足彩', u'现场荷官', 
                     u'真人现金网', u'网上赌场', u'赌钱网', u'现金赌博']

ZH_CATS["CEMETERY"] = [u'网上公墓,网上祭祀等服务', u'公墓', u'葬礼', u'陵园', u'墓地', u'安葬', u'平潭东方麒麟园', 
                       u'公益性公墓', u'经营性公墓', u'清明', u'骨灰', u'遗体', u'风水选址', u'圣地', 
                       u'皇陵', u'公共墓地', u'特色墓地', u'家族墓地', u'宗族墓地', u'古代诸侯王的公墓', 
                       u'合作公墓', u'教会公墓', u'公益性公墓', u'骨灰堂', u'抢风水', u'跟风水', u'航空烈士公墓', 
                       u'殡仪馆']

ZH_CATS["CROWD_FUNDING"] = [u'众筹', u'非法集资', u'人群资助运动', u'大众筹资', u'群众筹资', u'群众集资', 
                            u'群众募资', u'筹资项目', u'众筹平台', u'须弥山宗教众筹平台', u'宗教众筹', 
                            u'实物众筹', u'公益众筹', u'活动众筹', u'中国式游戏众筹']

ZH_CATS["CULT"] = [u'呼喊派', u'神的教会', u'地方召会 全范围教会', u'门徒会', u'三赎基督', u'蒙福派', 
                   u'二两粮', u'蒙头教', u'三赎教', u'灵灵教', u'被立王', u'全能神教会', 
                   u'全能神', u'东方闪电', u'实际神', u'七灵派', u'女基督派', u'观音法门', 
                   u'圆顿法门', u'中华大陆行政执事站', u'天父的儿女', u'达米宣教会', u'灵仙真佛宗', 
                   u'新约教会', u'世界以利亚福音宣教会', u'统一教', u'主神教', u'三班仆人派', 
                   u'中功', u'法轮功', u'圆顿法门', u'血水圣灵', u'华南教会', u'净宗学会', 
                   u'生命研究基金会', u'恩惠教', u'中华福音团契', u'静空学会', u'摩门教', 
                   u'摩尔门教', u'耶稣基督后期圣徒教会', u'徒弟会', u'旷野窄门', u'菩提功', 
                   u'耶和华见证人', u'白莲教', u'白莲社', u'华藏法门', u'华藏宗门', u'华藏玄门', 
                   u'人民圣殿教', u'人民寺院', u'奥姆真理教', u'天堂之门教', u'天门教', 
                   u'全世界高级计算机宗教组织', u'基督复临安息日会大卫支派', u'大卫支派', u'D-SDA']

ZH_CATS["DECEPTION"] = [u'替考,代考,代写论文服务', u'枪手', u'伪造', u'网络抢手', u'助考']

ZH_CATS["DONATION"] = [u'公益', u'基金会', u'慈善公益', u'捐款', u'慈善机构', u'慈善', u'壹基金', u'红十字会', 
                       u'捐助钱财', u'逃捐', u'假捐', u'募集', u'捐血', u'无偿献血', u'用血报销', u'捐赠', 
                       u'器官捐赠', u'捐助']

ZH_CATS["EXPLOSIVE"] = [u'煙花', u'烟花', u'爆竹', u'烟火', u'鞭炮', u'礼花', u'烟火', u'炮竹', 
                        u'礼炮', u'火药', u'礼花炮', u'西洋烟火', u'喷花类', u'旋转类', u'升空类', u'吐珠类', 
                        u'架子烟花', u'组合烟花']

ZH_CATS["FINANCIAL_SERVICES"] = [u'黄金投资', u'买卖银行账户', u'银行卡', u'股票', u'基金', u'保险', 
                                 u'保险平台', u'银行理财产品', u'返利', u'股票软件', u'股票资讯', 
                                 u'财经资讯', u'单用途预付卡', u'证券', u'外汇兑换服务', u'P2P金融互助平台', 
                                 u'POS支付', u'分期付款服务', u'虚开发票', u'虚拟货币', u'比特币', 
                                 u'莱特币', u'元宝币等虚拟货币交易', u'流通人民币', u'外币', u'融资', 
                                 u'储蓄', u'信贷', u'结算', u'证券买卖', u'商业保险', u'金融信息咨询', 
                                 u'直接保险', u'共同保险 ', u'寿险', u'非寿险', u'再保险', u'转分保', 
                                 u'保险中介', u'精算', u'风险评估', u'理赔服务', u'公众存款', u'消费信贷', 
                                 u'抵押信贷', u'商业交易的代理和融资', u'财务租赁', u'信用卡', u'赊账卡', 
                                 u'贷记卡', u'旅行支票', u'银行汇票', u'支票', u'汇票', u'存单', u'外汇', 
                                 u'衍生产品', u'期货', u'期权', u'汇率', u'利率', u'换汇', u'远期利率协议', 
                                 u'可转让证券', u'金银条块', u'结算和清算服务', u'证券']

ZH_CATS["FOREIGN_DATING"] = [u'涉外婚介', u'涉外婚姻', u'涉外结婚', u'涉外离婚', u'跨国婚姻', u'跨国结婚', u'国际婚姻']

ZH_CATS["FORGERY"] = [u'假币', u'违法办证刻章', u'假报告', u'伪造', u'变造', u'挖补', u'揭层', u'涂改', u'拼凑', 
                      u'移位', u'重印', u'网络假币']

ZH_CATS["GENDER_IDENTIFICATION"] = [u'胎儿性别鉴定', u'胎儿性别预测', u'性别鉴定', u'DNA母血鉴定', 
                                    u'羊水穿刺', u'绒毛采检', u'B超', u'Y染色体', u'抽绒毛鉴定', u'超音波', 
                                    u'宫颈黏液', u'胎心率']

ZH_CATS["IDENTITY_THEFT"] = [u'身份证信息', u'身份识别', u'银行账户', u'身份盗窃', u'非法获取', u'帐号', 
                             u'密码', u'社会保障识别号码', u'驾驶执照号码', u'信用卡号码']

ZH_CATS["MEDICAL_SERVICES"] = [u'医疗服务', u'医疗咨询', u'催眠', u'整容整形', u'医疗设施', u'医护人才', u'医保资金', 
                               u'疫病控制', u'妇幼保健', u'健康教育', u'卫生监督', u'医疗美容', u'医美', 
                               u'水光针', u'玻尿酸', u'隆鼻', u'隆胸']

ZH_CATS["MLM"] = [u'传销', u'多层次直销', u'庞氏骗局', u'层压式推销', u'金字塔销售', u'多层次信息网络营销', 
                  u'直销', u'金字塔骗局', u'层压式推销', u'层压式传销', u'连锁信', u'特许加盟经营', 
                  u'自愿连锁经营', u'网络资本运作', u'市场营销', u'连锁销售', u'纯资本运作', 
                  u'民间互助理财', u'人际网络', u'原始股基金', u'网络营销', u'框架营销', 
                  u'1040阳光工程', u'纯资本运作', u'人力资源连锁业', u'北部湾建设', u'点击广告获利', 
                  u'爱心互助', u'消费养老', u'境外基金', u'原始股投资', u'电子币买卖', u'致富梦想', u'新资本']

ZH_CATS["OIL"] = [u'石油', u'原油', u'汽油', u'煤油', u'化石燃料', u'岩石油', u'化石油', u'机油', u'矿物油', 
                  u'燃油', u'煤油', u'苯', u'汽油', u'航空汽油', u'车用汽油', u'喷气燃料', u'航空煤油', 
                  u'柴油机燃料', u'高速', u'中速', u'低速柴油', u'液态烃', u'炉用燃料油', u'船舶用燃料油', 
                  u'燃料', u'润滑油', u'溶溶剂']

ZH_CATS["PAWN"] = [u'典当', u'当铺', u'在线典当店', u'典当店', u'质铺', u'偿还当金', u'赎回当物', 
                   u'抵押', u'典当行', u'当铺', u'当票', u'死挡', u'当金', u'票号', u'钱庄', 
                   u'地下钱庄', u'交当', u'收当', u'存当', u'续当', u'当物查验', u'绝当']

ZH_CATS["PAYMENT_SERVICES"] = [u'付款服务', u'支付平台', u'财付通', u'交易中介', u'第三方电子支付平台', 
                               u'网上银行', u'电子信用卡', u'国付宝', u'易宝支付', u'快钱', u'百付宝', 
                               u'网易宝', u'环迅支付', u'汇付天下', u'好支付导航网', u'银联电子支付']

ZH_CATS["PENNY"] = [u'拍卖', u'互联网拍卖', u'网路拍卖 ', u'拍卖', u'招投标', u'拍卖商', u'卖出', 
                    u'竞价', u'成交价', u'竞购', u'标底', u'荷兰式拍卖', u'竞拍', u'英格兰式拍卖', 
                    u'密封递价式', u'标准增量式', u'维克瑞式', u'速胜式', u'反向拍卖', u'定向拍卖', 
                    u'拍定物', u'古董拍卖', u'古玩拍卖', u'玉器拍卖']

ZH_CATS["PHARMA"] = [u'毒品', u'吸毒工具', u'剧毒物品和危险化学品', u'麻醉药品', u'精神类药品', u'处方药', u'无批号药品', 
                     u'成人药品', u'洋地黄毒苷片', u'硫酸阿托品片', u'甲溴阿托品片', u'硝酸毛果芸香碱片', 
                     u'异戊巴比妥片', u'安钠咖片', u'盐酸芬氟拉明片', u'丁丙诺啡舌下含片', u'阿普唑仑片', 
                     u'阿普唑仑胶囊', u'氯氮卓片', u'氯硝西泮片', u'地西泮片', u'地西泮膜', u'艾司唑仑片', 
                     u'氯氟卓乙酯片', u'单盐酸氟西泮胶囊', u'劳拉西泮片', u'甲丙氨酯片', u'马来酸咪达唑仑片', 
                     u'硝西泮片', u'匹莫林片', u'苯巴比妥片', u'酒石酸唑吡坦片', u'酒石酸唑吡坦颗粒', 
                     u'扎来普隆片', u'扎来普隆分散片', u'扎来普隆胶囊', u'麦角胺咖啡因片', u'盐酸司来吉兰片', 
                     u'盐酸司来吉兰胶囊', u'乙酰唑胺片', u'盐酸阿米洛利片', u'阿替洛尔片', u'苄氟噻嗪片', 
                     u'盐酸倍他洛尔片', u'盐酸倍他洛尔滴眼液', u'富马酸比索洛尔片', u'富马酸比索洛尔胶囊', 
                     u'布美他尼片', u'卡维地洛片', u'卡维地洛胶囊', u'盐酸塞利洛尔片', u'盐酸塞利洛尔胶囊', 
                     u'氯噻酮片', u'依他尼酸片', u'富马酸福莫特罗片', u'富马酸福莫特罗干糖浆', 
                     u'富马酸福莫特罗粉吸入剂', u'呋塞米片', u'氢氯噻嗪片', u'吲达帕胺胶囊', u'吲达帕胺缓释胶囊', 
                     u'吲达帕胺片', u'吲达帕胺缓释片', u'吲达帕胺滴丸', u'盐酸拉贝洛尔片', u'盐酸左布诺洛尔滴眼液', 
                     u'美替洛尔滴眼剂', u'酒石酸美托洛尔片', u'酒石酸美托洛尔控释片', u'酒石酸美托洛尔胶囊', 
                     u'酒石酸美托洛尔缓释片', u'氧烯洛尔片', u'吲哚洛尔片', u'丙磺舒片', u'盐酸普萘洛尔片', 
                     u'盐酸普萘洛尔缓释片', u'盐酸普萘洛尔缓释胶囊', u'盐酸左旋沙丁胺醇吸入用溶液', 
                     u'硫酸沙丁胺醇口腔崩解片', u'硫酸沙丁胺醇片', u'硫酸沙丁胺醇胶囊', u'硫酸沙丁胺醇缓释片', 
                     u'硫酸沙丁胺醇控释胶囊', u'盐酸索他洛尔片', u'螺内酯片', u'螺内酯胶囊', u'硫酸特布他林片', 
                     u'硫酸特布他林口服溶液', u'硫酸特布他林颗粒', u'硫酸特布他林干糖浆', u'硫酸特布他林气雾剂', 
                     u'硫酸特布他林吸入粉雾剂', u'马来酸噻吗洛尔滴眼液', u'氨苯蝶啶片', u'阿那曲唑片', 
                     u'依西美坦片', u'依西美坦胶囊', u'枸橼酸托瑞米芬片', u'"盐酸氯丙嗪片', u'奋乃静片', 
                     u'盐酸氟奋乃静片', u'盐酸三氟拉嗪片', u'盐酸硫利达嗪片', u'氟哌啶醇片', u'五氟利多片', 
                     u'盐酸三氟哌多片', u'氯普噻吨片', u'珠氯噻醇片', u'二盐酸珠氯噻醇片', u'氟哌噻吨片', 
                     u'二盐酸氟哌噻吨片', u'氟哌噻吨美利曲辛片', u'舒必利片', u'盐酸硫必利片', u'利培酮片', 
                     u'利培酮口崩片', u'利培酮口服液', u'氯氮平片', u'奥氮平片', u'丁二酸洛沙平胶囊', 
                     u'盐酸舒托必利片', u'盐酸氯丙咪嗪片', u'盐酸氯米帕明片', u'盐酸阿米替林片', u'盐酸马普替林片', 
                     u'盐酸米安色林片', u'吗氯贝胺胶囊', u'马来酸氟伏沙明片', u'盐酸氟西汀片', u'盐酸氟西汀胶囊', 
                     u'盐酸帕罗西汀片', u'盐酸舍曲林片', u'盐酸曲唑酮片', u'硫酸苯丙胺片', u'盐酸多塞平片', 
                     u'盐酸多塞平乳膏', u'异卡波肼片', u'盐酸文拉法辛胶囊', u'盐酸安非他酮片', u'盐酸安非他酮缓释片', 
                     u'"硫酸阿巴卡韦片', u'硫酸阿巴卡韦口服溶液', u'阿巴卡韦双夫定片4去羟肌苷咀嚼片', u'去羟肌苷分散片', 
                     u'去羟肌苷颗粒', u'去羟肌苷肠溶胶囊', u'', u'去羟肌苷散', u'拉米夫定片', u'拉米夫定口服溶液', 
                     u'齐多夫定', u'拉米夫定片', u'司他夫定片', u'司他夫定胶囊', u'齐多夫定片', u'齐多夫定胶囊', 
                     u'齐多夫定口服溶液', u'奈韦拉平片', u'奈韦拉平胶囊', u'奈韦拉平口服混悬液', u'硫酸茚地那韦片', 
                     u'硫酸茚地那韦胶囊', u'利托那韦口服溶液', u'利托那韦软胶囊', u'洛匹那韦', u'利托那韦口服液', 
                     u'洛匹那韦', u'利托那韦软胶囊', u'甲磺酸沙奎那韦胶囊', u'沙奎那韦胶丸', u'"盐酸氮芥搽剂', 
                     u'盐酸氮芥酊', u'苯丁酸氮芥纸型片', u'环磷酰胺片', u'复方环磷酰胺片', u'氮甲片', 
                     u'甘磷酰芥片', u'美法仑片', u'洛莫司汀胶囊', u'司莫司汀胶囊', u'苯丁酸氮芥纸型片', 
                     u'六甲蜜胺片', u'六甲蜜胺肠溶片', u'六甲蜜胺胶囊', u'白消安片', u'异芳芥片', u'甲氨蝶呤片', 
                     u'巯嘌呤片', u'氟尿嘧啶片', u'氟尿嘧啶口服乳', u'氟尿嘧啶软膏', u'复方氟尿嘧啶口服溶液', 
                     u'替加氟片', u'替加氟胶囊', u'替加氟栓', u'尿嘧啶替加氟片', u'尿嘧啶替加氟胶囊', u'卡培他滨片', 
                     u'卡培他滨颗粒羟基脲片', u'羟基脲胶囊', u'植入用缓释依托泊苷', u'依托泊苷胶囊', 
                     u'枸橼酸他莫昔芬片', u'氨鲁米特片', u'来曲唑片', u'醋酸甲羟孕酮片', u'醋酸甲羟孕酮分散片', 
                     u'醋酸甲羟孕酮复合胶囊', u'醋酸甲地孕酮片', u'醋酸甲地孕酮分散片', u'复方醋酸甲地孕酮片', 
                     u'盐酸丙卡巴肼肠溶片', u'甲磺酸伊马替尼胶囊去甲斑蝥素片', u'维A酸片45维A酸胶囊', 
                     u'重组人干扰素α1a滴眼液', u'重组人干扰素α2b栓剂', u'盐酸昂丹司琼片', u'盐酸昂丹司琼胶囊', 
                     u'盐酸格拉司琼片', u'盐酸托烷司琼片', u'盐酸托烷司琼胶囊', u'香菇多糖片', u'复方氢可酮溶液', 
                     u'复方氢可酮止咳溶液', u'氢可酮复方镇咳糖浆', u'可愈糖浆', u'5复方磷酸可待因溶液', 
                     u'复方磷酸可待因糖浆7复方磷酸可待因口服溶液', u'复方福尔可定口服液', u'复方福尔可定糖浆', 
                     u'复方甘草口服溶液', u'盐酸曲马多片', u'盐酸曲马多缓释片', u'盐酸曲马多控释片', 
                     u'盐酸曲马多泡腾片', u'盐酸曲马多分散片', u'盐酸曲马多泡腾颗粒', u'盐酸曲马多胶囊', 
                     u'盐酸曲马多缓释胶囊', u'盐酸曲马多滴剂', u'盐酸曲马多栓', u'复方曲马多片', u'氨酚曲马多片', 
                     u'氢化可的松片', u'醋酸氢化可的松片', u'醋酸氢化可的松滴眼液', u'复方氢化可的松胶布', 
                     u'醋酸泼尼松片', u'醋酸泼尼松眼膏', u'四环素泼尼松眼膏', u'复方四环素泼尼松膜', u'泼尼松龙片', 
                     u'醋酸泼尼松龙片', u'甲泼尼龙片', u'曲安西龙片', u'复方醋酸曲安西龙涂剂', u'曲安奈德新霉素贴膏', 
                     u'曲安奈德氯霉素溶液', u'复方醋酸曲安奈德滴耳液', u'复方曲安奈德乳膏', 
                     u'复方十一烯酸锌曲安奈德软膏', u'布地奈德气雾剂', u'布地奈德', u'福莫特罗复方粉雾吸入剂', 
                     u'吸入用布地奈德混悬液', u'丙酸氟替卡松气雾剂', u'沙美特罗', u'丙酸氟替卡松干粉吸入剂', 
                     u'丙酸氟替卡松鼻喷雾剂', u'丙酸氟替卡松粉雾剂', u'丙酸氟替卡松软膏', u'糠酸莫米松鼻喷雾剂', 
                     u'地塞米松片', u'醋酸地塞米松片', u'地塞米松磷酸钠滴眼液', u'醋酸地塞米松软膏', u'倍他米松片', 
                     u'倍他米松软膏', u'醋酸倍他米松搽剂', u'克霉唑倍他米松乳膏', u'醋酸氟氢可的松软膏', 
                     u'丙酸氯倍他索乳膏', u'咪康唑氯倍他索乳膏', u'醋酸氟轻松软膏', u'醋酸氟轻松搽剂', 
                     u'冰樟桉氟轻松贴膏', u'新霉素氟轻松乳膏', u'氟轻松维B6乳膏', u'醋酸氟轻松冰片乳膏', 
                     u'复方醋酸氟轻松酊', u'复方氟轻松尿素膏', u'丙酸倍氯米松气雾剂', u'丙酸倍氯米松软膏', 
                     u'复方倍氯米松新霉素贴膏', u'哈西奈德溶液', u'哈西奈德软膏', u'哈西奈德乳膏', 
                     u'醋酸可的松片', u'氟米龙滴眼液', u'庆大霉素氟米龙滴眼液', u'卤米松乳膏', u'卤米松', 
                     u'三氯生软膏', u'复方八维甲睾酮胶囊', u'雌二醇缓释贴片', u'炔雌醇片', u'炔诺孕酮炔雌醚片', 
                     u'雌三醇栓', u'雌三醇软膏', u'尼尔雌醇片', u'己烯雌酚片', u'己二烯雌酚片', 
                     u'氯烯雌醚滴丸', u'黄体酮栓', u'醋酸甲羟孕酮片', u'醋酸甲羟孕酮分散片', 
                     u'醋酸甲羟孕酮复合胶囊', u'复方炔雌醇甲羟孕酮胶囊', u'炔孕酮片', u'醋酸环丙孕酮片', 
                     u'枸橼酸氯米芬片', u'枸橼酸氯米芬胶囊', u'醋酸戈舍瑞林缓释植入剂', u'炔诺酮片', 
                     u'醋酸甲地孕酮片', u'醋酸甲地孕酮分散片', u'炔诺孕酮炔雌醚片', u'左炔诺孕酮炔雌醚片', 
                     u'双炔失碳酯片', u'复方双炔失碳酯肠溶片', u'复方醋酸棉酚片', u'棉酚氯化钾维B胶囊', 
                     u'甲状腺素碘塞罗宁滴眼液', u'左甲状腺素钠片', u'鲑鱼降钙素喷鼻剂', u'丙硫氧嘧啶片', 
                     u'甲巯咪唑片', u'卡比马唑片', u'碘化钾片', u'	阿米卡星滴眼液', u'阿米卡星洗剂', 
                     u'阿莫西林分散片', u'阿莫西林干混悬剂', u'阿莫西林胶囊', u'阿莫西林咀嚼片', 
                     u'阿莫西林颗粒', u'阿莫西林克拉维酸钾干混悬剂', u'阿莫西林克拉维酸钾颗粒', 
                     u'阿莫西林克拉维酸钾片', u'阿莫西林片（羟氨苄青霉素片)', u'阿莫西林舒巴坦匹酯片', 
                     u'阿莫西林舒巴坦片', u'阿奇霉素分散片', u'阿奇霉素干混悬剂', u'阿奇霉素胶囊', 
                     u'阿奇霉素颗粒剂', u'阿奇霉素片', u'阿奇霉素细粒剂', u'氨苄西林胶囊', 
                     u'氨苄西林颗粒(氨苄西林干混悬剂)', u'氨苄西林片', u'苯唑西林钠胶囊', 
                     u'苯唑西林钠片', u'丙酸交沙霉素干糖浆', u'丙酸交沙霉素颗粒剂', 
                     u'醋酸麦迪霉素颗粒剂', u'单硫酸卡那霉素颗粒', u'地红霉素肠溶胶囊', 
                     u'地红霉素肠溶片', u'二维四环素甲氧苄啶片', u'氟氯西林钠胶囊', u'氟氯西林片', 
                     u'复方硫酸新霉素滴眼液', u'复方氯霉素栓', u'复方氯霉素阴道泡腾片', 
                     u'复方氢化可的松新霉素乳膏', u'复方庆大霉素膜', u'复方四环素片', u'富马酸阿奇霉素片', 
                     u'海他西林钾片', u'红霉素肠溶胶囊', u'红霉素肠溶片', u'红霉素肠溶散剂', u'琥乙红霉素分散片', 
                     u'琥乙红霉素咀嚼片', u'琥乙红霉素颗粒', u'琥乙红霉素片', u'环酯红霉素片', 
                     u'甲苯磺酸舒他西林胶囊', u'甲苯磺酸舒他西林颗粒剂', u'甲苯磺酸舒他西林片', u'甲砜霉素胶囊', 
                     u'甲砜霉素颗粒', u'甲砜霉素片', u'交沙霉素胶囊', u'交沙霉素颗粒剂', u'克拉霉素分散片', 
                     u'克拉霉素干混悬剂', u'克拉霉素缓释片', u'克拉霉素胶囊', u'克拉霉素颗粒剂', u'克拉霉素片', 
                     u'林可霉素维B6乳膏', u'磷霉素氨丁三醇散', u'磷霉素钙甲氧苄啶胶囊', u'磷霉素钙胶囊', 
                     u'磷霉素钙颗粒', u'硫酸巴龙霉素片', u'硫酸卡那霉素滴眼液', u'硫酸卡那霉素胶囊', u'硫酸卡那霉素片', 
                     u'硫酸卡那霉素软膏', u'硫酸庆大霉素二氧化锆缓释丸链', u'硫酸庆大霉素缓释片', 
                     u'硫酸庆大霉素碱式硝酸铋胶囊', u'硫酸庆大霉素胶囊', u'硫酸庆大霉素咀嚼片', u'硫酸庆大霉素颗粒', 
                     u'硫酸庆大霉素口服液', u'硫酸庆大霉素片', u'硫酸小诺霉素滴眼液', u'硫酸小诺霉素干糖浆', 
                     u'硫酸小诺霉素口服液', u'硫酸小诺霉素片', u'硫酸新霉素滴眼液', u'硫酸新霉素片', u'硫酸粘菌素颗粒', 
                     u'硫酸粘菌素片', u'氯霉素搽剂', u'氯霉素耳栓', u'氯霉素耳丸', u'氯霉素胶囊', 
                     u'氯霉素控释眼丸', u'氯霉素片', u'氯霉素软胶囊', u'氯霉素眼膏', u'氯唑西林钠胶囊', 
                     u'氯唑西林钠颗粒', u'罗红霉素分散片', u'罗红霉素干混悬剂', u'罗红霉素胶囊', 
                     u'罗红霉素颗粒', u'罗红霉素片', u'罗红霉素细粒剂', u'螺旋霉素片', u'麦迪霉素胶囊', 
                     u'麦迪霉素片', u'匹美西林胶囊', u'匹美西林片', u'青霉素V钾分散片', u'青霉素V钾胶囊', 
                     u'青霉素V钾颗粒', u'青霉素V钾口服液', u'青霉素V钾混悬剂', u'青霉素V钾片', 
                     u'氢化可的松新霉素滴耳液', u'庆大霉素氟米龙滴眼液', u'庆大霉素普鲁卡因胶囊', 
                     u'庆大霉素双氯芬酸钠滴眼液', u'庆大霉素碳酸铋胶囊1曲安奈德新霉素贴膏', u'乳糖酸克拉霉素胶囊', 
                     u'舒他西林干混悬剂', u'双氯西林胶囊', u'四环素甲氧苄啶胶囊', u'泰利霉素片', 
                     u'头孢氨苄干混悬剂', u'头孢氨苄缓释胶囊', u'头孢氨苄缓释片', u'头孢氨苄甲氧苄啶胶囊', 
                     u'头孢氨苄甲氧苄啶颗粒', u'头孢氨苄胶囊', u'头孢氨苄颗粒', u'头孢氨苄泡腾片', 
                     u'头孢氨苄片', u'头孢泊肟匹酯片', u'头孢泊肟酯干混悬剂', u'头孢泊肟酯片', 
                     u'头孢地尼胶囊', u'头孢地尼颗粒', u'头孢呋辛酯分散片', u'头孢呋辛酯干混悬剂', 
                     u'头孢呋辛酯胶囊', u'头孢呋辛酯片', u'头孢克罗泡腾片', u'头孢克洛分散片', 
                     u'头孢克洛干混悬剂', u'头孢克洛缓释胶囊', u'头孢克洛缓释片', u'头孢克洛胶囊', 
                     u'头孢克洛颗粒', u'头孢克洛片', u'头孢克肟分散片', u'头孢克肟胶囊', 
                     u'头孢克肟颗粒', u'头孢克肟片', u'头孢拉定干混悬剂', u'头孢拉定胶囊', 
                     u'头孢拉定颗粒', u'头孢拉定片', u'头孢羟氨苄甲氧苄啶胶囊', u'头孢羟氨苄胶囊',
                     u'头孢羟氨苄颗粒', u'头孢羟氨苄片', u'头孢沙定胶囊', u'头孢特仑新戊酯片', 
                     u'头孢妥仑匹酯片', u'土霉素软膏', u'托西酸舒他西林片', u'妥布霉素滴眼液', 
                     u'小儿用阿奇霉素干混悬剂', u'新地松眼膏', u'新霉素氟轻松乳膏', u'盐酸巴氨西林片', 
                     u'盐酸巴坎西林片', u'盐酸多西环素肠溶胶囊', u'盐酸多西环素胶囊', u'盐酸多西环素片', 
                     u'盐酸胍甲环素胶囊', u'盐酸胍甲环素片', u'盐酸克林霉素胶囊', u'盐酸克林霉素溶液', 
                     u'盐酸克林霉素棕榈酸酯分散片', u'盐酸克林霉素棕榈酸酯颗粒剂', u'盐酸林可霉素滴耳液', 
                     u'盐酸林可霉素滴眼液', u'盐酸林可霉素胶囊', u'盐酸林可霉素口服溶液', u'盐酸林可霉素片', 
                     u'盐酸林可霉素溶液', u'盐酸林可霉素软膏', u'盐酸林可霉素栓', u'盐酸美他环素胶囊', 
                     u'盐酸美他环素片', u'盐酸米诺环素胶囊', u'盐酸米诺环素片', u'盐酸米诺环素软膏', 
                     u'盐酸米诺环素牙用缓释膜', u'盐酸莫西沙星片', u'盐酸四环素胶囊', u'盐酸四环素片', 
                     u'盐酸头孢他美酯干混悬剂', u'盐酸头孢他美酯胶囊', u'盐酸头孢他美酯片', u'盐酸土霉素胶囊', 
                     u'盐酸土霉素片', u'依托红霉素胶囊', u'依托红霉素颗粒', u'依托红霉素片', 
                     u'乙酰吉他霉素干糖浆', u'乙酰吉他霉素含片', u'乙酰吉他霉素胶囊', u'乙酰吉他霉素颗粒', 
                     u'乙酰螺旋霉素胶囊', u'乙酰螺旋霉素片', u'乙酰麦迪霉素干糖浆', u'乙酰麦迪霉素颗粒剂', 
                     u'乙酰麦迪霉素片', u'硬脂酸红霉素胶囊', u'硬脂酸红霉素颗粒', u'硬脂酸红霉素片', 
                     u'棕榈氯霉素颗粒', u'棕榈氯霉素片', u'棕榈氯霉素混悬液', u'颠茄磺苄啶片', 
                     u'复方磺胺对甲氧嘧啶片', u'复方磺胺甲噁唑分散片', u'复方磺胺甲噁唑混悬液', 
                     u'复方磺胺甲噁唑胶囊', u'复方磺胺甲噁唑颗粒', u'复方磺胺甲噁唑口服混悬液', 
                     u'复方磺胺甲噁唑片', u'复方磺胺脒片', u'复方磺胺嘧啶颗粒', u'复方磺胺嘧啶片', 
                     u'复方磺胺氧化锌软膏', u'琥磺胺噻唑片', u'磺胺苯吡唑片', u'磺胺多辛片', 
                     u'磺胺二甲嘧啶干混悬剂', u'磺胺二甲嘧啶混悬液', u'磺胺二甲嘧啶片', 
                     u'磺胺二甲异噁唑片(菌得清片)', u'磺胺甲噁唑片', u'磺胺间甲氧嘧啶片', u'磺胺林片', 
                     u'磺胺脒片', u'磺胺嘧啶混悬液', u'磺胺嘧啶片', u'磺胺嘧啶软膏', u'磺胺嘧啶速释片', 
                     u'磺胺嘧啶锌软膏', u'磺胺嘧啶眼膏', u'磺胺索嘧啶片（磺胺二甲异嘧啶片)', 
                     u'磺胺氧化锌软膏', u'磺胺异噁唑片', u'磺啶冰黄片', u'磺啶新林胶囊', 
                     u'磺啶新林颗粒', u'联磺甲氧苄啶胶囊', u'联磺甲氧苄啶片', u'柳氮磺吡啶片', 
                     u'柳氮磺吡啶栓', u'双磺沙棘桉青软膏', u'酞磺胺噻唑片', u'酞磺醋胺片', 
                     u'小儿复方磺胺二甲嘧啶散', u'小儿复方磺胺甲噁唑颗粒', u'小儿复方磺胺甲噁唑片', 
                     u'小儿复方磺胺甲噁唑散', u'小儿复方磺胺嘧啶颗粒', u'小儿枸磺新啶片', 
                     u'小儿双磺甲氧苄啶颗粒', u'小儿双磺甲氧苄啶散', u'小儿双嘧啶颗粒', u'小儿双嘧啶片', 
                     u'吡哌酸滴丸', u'吡哌酸胶囊', u'吡哌酸颗粒', u'吡哌酸片', u'吡哌酸锌软膏', 
                     u'氟罗沙星胶囊', u'氟罗沙星片', u'环丙沙星片', u'加替沙星胶囊', u'加替沙星片', 
                     u'甲苯磺酸托氟沙星胶囊', u'甲磺酸培氟沙星胶囊', u'甲磺酸培氟沙星片', 
                     u'甲磺酸培氟沙星软膏', u'甲磺酸左氧氟沙星胶囊', u'甲磺酸左氧氟沙星片', 
                     u'莫昔沙星片', u'萘啶酸片', u'诺氟沙星滴眼液', u'诺氟沙星胶囊', u'诺氟沙星片', 
                     u'诺氟沙星软膏', u'诺氟沙星栓', u'诺氟沙星锌乳膏', u'诺氟沙星锌散', 
                     u'乳酸氟罗沙星片', u'乳酸环丙沙星滴眼液', u'乳酸司氟沙星片', u'乳酸左氧氟沙星滴眼乳酸左氧氟沙星胶囊', 
                     u'乳酸左氧氟沙星片', u'司氟沙星片', u'司帕沙星分散片', u'司帕沙星胶囊', u'司帕沙星片', 
                     u'盐酸环丙沙星滴耳液', u'盐酸环丙沙星滴眼液', u'盐酸环丙沙星胶囊', u'盐酸环丙沙星颗粒剂', 
                     u'盐酸环丙沙星片', u'盐酸环丙沙星乳膏', u'盐酸环丙沙星软膏', u'盐酸环丙沙星栓', 
                     u'盐酸环丙沙星眼膏', u'盐酸环丙沙星阴道泡腾片', u'盐酸加替沙星片', 
                     u'盐酸芦氟沙星胶囊48盐酸芦氟沙星片', u'盐酸洛美沙星滴耳液', u'盐酸洛美沙星滴眼液', 
                     u'盐酸洛美沙星胶囊', u'盐酸洛美沙星颗粒', u'盐酸洛美沙星片', u'盐酸洛美沙星乳膏', 
                     u'盐酸洛美沙星眼用凝胶', u'盐酸左氧氟沙星滴眼液', u'盐酸左氧氟沙星胶囊', u'盐酸左氧氟沙星片', 
                     u'氧氟沙星滴耳液', u'氧氟沙星滴眼液', u'氧氟沙星胶囊', u'氧氟沙星颗粒', 
                     u'氧氟沙星凝胶', u'氧氟沙星片', u'氧氟沙星乳膏', u'氧氟沙星软膏', u'氧氟沙星栓', 
                     u'氧氟沙星眼膏', u'氧氟沙星阴道泡腾片', u'依诺沙星滴眼液', u'依诺沙星胶囊', 
                     u'依诺沙星片', u'左氧氟沙星片', u'安痨息片', u'氨硫脲片', u'吡嗪酰胺胶囊', 
                     u'吡嗪酰胺片', u'丙硫异烟胺片', u'对氨基水杨酸钠肠溶片', u'利福定胶囊', 
                     u'利福定片', u'利福喷丁胶囊', u'利福喷丁片', u'利福平滴眼液', u'利福平胶囊', 
                     u'利福平胶丸', u'利福平片', u'利福平眼膏', u'氯法齐明胶丸', u'帕司烟肼片', 
                     u'盐酸乙胺丁醇胶囊', u'盐酸乙胺丁醇片', u'乙硫异烟胺肠溶片', u'异福片', 
                     u'异福酰胺胶囊', u'异福酰胺片', u'异烟肼片', u'异烟腙片', u'氟胞嘧啶片', 
                     u'氟康唑滴眼液', u'氟康唑胶囊', u'氟康唑颗粒剂', u'氟康唑片', 
                     u'氟康唑气雾剂（混悬型)', u'复方灰黄霉素搽剂', u'复方咪康唑曲安奈德乳膏', 
                     u'复方珊瑚姜溶液尿素咪康唑软膏复合制剂', u'复方酮康唑乳膏', u'环吡酮胺栓', 
                     u'灰黄霉素胶囊', u'灰黄霉素片', u'康松无极膏', u'克霉唑尿素乳膏', 
                     u'克霉唑片', u'克念菌素片', u'联苯苄唑栓', u'两性霉素B滴眼剂', u'两性霉素B栓剂', 
                     u'两性霉素B阴道泡腾片', u'美帕曲星肠溶片', u'美帕曲星阴道片', u'咪康唑氯倍他索乳膏', 
                     u'灭癣酚溶液', u'酮康他索乳膏（otc)', u'酮康唑混悬液', u'酮康唑胶囊', u'酮康唑片', 
                     u'托萘酯乳膏', u'托萘酯软膏', u'西卡宁油剂', u'西卡宁酊剂', 
                     u'硝呋太尔－制霉菌素阴道栓', u'硝酸咪康唑胶囊', u'盐酸阿莫罗芬搽剂', 
                     u'盐酸布替萘芬喷剂', u'盐酸萘替芬溶液', u'盐酸特比萘芬片', u'伊曲康唑胶囊',
                     u'伊曲康唑口服液', u'伊曲康唑片', u'益康倍松乳膏', u'制霉菌素片', u'注射剂']

ZH_CATS["PROPERTY"] = [u'不动产买卖', u'房产', u'土地', u'房屋', u'探矿权', u'采矿权']

ZH_CATS["PROPAGANDA"] = [u'非法政治音像制品', u'出版物', u'非法反动卡类和节目台', u'宗教网站', 
                         u'卫星天线', u'犹太教', u'佛教', u'伊斯兰教', u'基督教', 
                         u'道教']

ZH_CATS["RESTRICTED_ITEMS"] = [u'军用', u'警用物品', u'管制器具', u'开锁工具', u'间谍器材', u'警察背心', 
                               u'战术背心', u'攻击背心', u'防弹背心', u'对讲机', u'手铐', u'警棍', 
                               u'眩晕枪', u'脚铐', u'刺背心', u'警察头盔', u'雷达枪', u'警察盾牌', 
                               u'弹药袋', u'防弹背心', u'电枪']

ZH_CATS["SEEDS"] = [u'种子', u'农业种子', u'裸子植物', u'被子植物', u'繁殖体', u'胚珠', u'传粉受精', 
                    u'种皮', u'胚', u'胚乳', u'孢子体胚珠', u'蕨类植物', u'休眠现象', u'晒种', u'浸种', 
                    u'催芽', u'自体传播', u'风力传播', u'水力传播', u'鸟类传播', u'哺乳动物传播', 
                    u'昆虫传播', u'昆虫传播', u'虫媒花', u'风力传播', u'风媒花', u'蒲公英', 
                    u'黑板树', u'昭和草', u'鬼针草', u'雀榕', u'车前草', u'非洲凤仙', u'羊蹄甲', u'洋紫荆']

ZH_CATS["SMOKE"] = [u'烟草', u'香烟', u'抽烟', u'电子烟', u'烟袋', u'旱烟', u'烟叶', u'万宝路', u'电子烟油', 
                    u'水烟', u'鼻烟', u'凉烟', u'电子烟液']

ZH_CATS["SURROGACY"] = [u'代孕服务', u'代孕', u'代孕妈妈', u'卵供体', u'商业代母', u'人类辅助生殖技术', 
                        u'不育患者', u'体外受精', u'试管婴儿IVF', u'人工授精', u'局部代孕', u'夫精人工授精', 
                        u'供精人工授精', u'混精人工授精', u'试管婴儿代孕', u'胚胎移植', u'排卵障碍', 
                        u'子宫内膜异位症', u'免疫性不孕', u'生精功能障碍', u'免疫性不育', u'自然代孕', 
                        u'冷冻储存胚胎', u'同居代孕', u'捐卵', u'代孕妈妈', u'有偿代孕']

ZH_CATS["WEAPON"] = [u'左轮手枪', u'发射器', u'火箭发射器', u'步枪', u'手枪', u'猎枪', u'狙击枪', 
                     u'长步枪', u'机枪', u'加特林机枪', u'冲锋枪', u'半自动手枪', u'突击步枪', 
                     u'加特林枪', u'螺栓动作卡宾枪', u'驱逐舰马枪', u'迫击炮弹', u'反物资步枪', 
                     u'单发榴弹发射器', u'榴弹发射器', u'猎枪', u'榴弹炮', u'弹道式导弹', u'导弹', 
                     u'反坦克炮', u'迫击炮', u'手榴弹', u'手枪', u'喷火器', u'战斗刀', u'指节铜环', 
                     u'匕首', u'短剑', u'双节棍', u'矛', u'戟', u' 标枪', u'弩', u'长弓', u'燧发枪', 
                     u'槊', u'剑', u'电棍', u'匕首刀', u'兵器', u'弓', u'箭', u'刀', u'枪', u'火器', 
                     u'军火', u'军械', u'军器', u'冲火炮', u'毒气', u'战列舰', u'坦克', u'飞机', u'航母', 
                     u'潜舰', u'特种枪', u'散弹枪', u'加农炮', u'火箭炮', u'迫击炮', u'高射炮', u'坦克炮', 
                     u'反坦克炮', u'航空炮', u'舰炮', u'海岸炮', u'坦克', u'装甲输送车', u'步兵战车', 
                     u'战斗舰艇', u'航空母舰', u'战列舰', u'巡洋舰', u'驱逐舰', u'护卫舰', u'潜艇', 
                     u'导弹', u'两栖作战舰艇', u'两栖攻击舰', u'两栖运输舰', u'登陆舰艇', u'勤务舰艇', 
                     u'侦察舰船', u'抢险救生舰船', u'航行补给舰船', u'训练舰', u'医院船', u'军用人造卫星', 
                     u'宇宙飞船', u'空间站', u'航天飞机', u'作战飞机', u'轰炸机', u'歼击机', u'强击机', 
                     u'反潜机', u'勤务飞机', u'侦察机', u'预警机', u'电子干扰机', u'空中加油机', u'教练机', 
                     u'直升机', u'武装直升机', u'空中运输直升机', u'无人驾驶飞机', u'军用飞艇', u'航空炸弹', 
                     u'火箭弹', u'导弹弹头', u'化学地雷', u'橡皮子弹', u'催泪瓦斯', u'炫目弹', 
                     u'高压水枪', u'生物战剂', u'细菌', u'毒素', u'真菌', u'航空炸弹', u'地雷', 
                     u'水雷', u'火炸药', u'原子弹', u'氢弹', u'中子弹', u'核弹头', u'制导导弹', 
                     u'制导炮弹', u'隐形飞机', u'隐形导弹', u'隐形舰船', u'隐形坦克', u'定向能武器', 
                     u'激光武器', u'微波武器', u'粒子束武器', u'动能武器', u'动能拦截弹', u'电磁炮', 
                     u'群射火箭', u'军用机器人', u'电脑“病毒']

ZH_CATS["VIDEO_CHAT"] = [u'视频聊天', u'直播', u'网络电话', u'网络钓鱼', u'钓鱼法', u'钓鱼式攻击', u'网络诈骗', 
                         u'网恋', u'网聊']

ZH_CATS["PORN"] = [u'黄色低俗音像制品', u'出版物', u'黄色低俗色情服务', u'黄色低俗卡类和节目台', u'成人影片',
                  u'A片', u'毛片', u'色情', u'情色', u'三级片', u'淫秽', u'淫乱', u'猥亵']

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Define the results output structure
</div>

In [18]:
results = { 
            "CASINO:" : casino_score,
            "CEMETERY:" : cemetery_score,
            "CROWD_FUNDING:" : crowd_funding_score,
            "CULT:" : cult_score,
            "DECEPTION:" : deception_score,
            "DONATION:" : donation_score,
            "EXPLOSIVE:" : explosive_score,
            "FINANCIAL SERVICES:" : financial_services_score,
            "FOREIGN DATING:" : foreign_dating_score,
            "FORGERY:" : forgery_score,
            "GENDER IDENTIFICATION:" : gender_identification_score,
            "IDENTITY THEFT:" : identity_theft_score,
            "MEDICAL SERVICES:" : medical_services_score,
            "MLM:" : mlm_score,
            "OIL:" : oil_score,
            "PAWN:" : pawn_score,
            "PAYMENT SERVICES:" : payment_services_score,
            "PENNY:" : penny_score,
            "PHARMA:" : pharma_score,
            "PROPERTY:" : property_score,
            "PROPAGANDA:" : propaganda_score,
            "RESTRICTED ITEMS:" : restricted_items_score,
            "SEEDS:" : seeds_score,
            "SMOKE:" : smoke_score,
            "SURROGACY:" : surrogacy_score,
            "WEAPON:" : weapon_score,
            "VIDEO CHAT:" : video_chat_score,
            "PORN:" : porn_score
}

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Define the counters for each of the categories
</div>

In [5]:
counter_casino = Counter(ZH_CATS["CASINO"])
counter_cemetery = Counter(ZH_CATS["CEMETERY"])
counter_crowd_funding = Counter(ZH_CATS["CROWD_FUNDING"])
counter_cult = Counter(ZH_CATS["CULT"])
counter_deception = Counter(ZH_CATS["DECEPTION"])
counter_donation = Counter(ZH_CATS["DONATION"])
counter_explosive = Counter(ZH_CATS["EXPLOSIVE"])
counter_financial_services = Counter(ZH_CATS["FINANCIAL_SERVICES"])
counter_foreign_dating = Counter(ZH_CATS["FOREIGN_DATING"])
counter_forgery = Counter(ZH_CATS["FORGERY"])
counter_gender_identification = Counter(ZH_CATS["GENDER_IDENTIFICATION"])
counter_identity_theft = Counter(ZH_CATS["IDENTITY_THEFT"])
counter_medical_services = Counter(ZH_CATS["MEDICAL_SERVICES"])
counter_mlm = Counter(ZH_CATS["MLM"])
counter_oil = Counter(ZH_CATS["OIL"])
counter_pawn = Counter(ZH_CATS["PAWN"])
counter_payment_services = Counter(ZH_CATS["PAYMENT_SERVICES"])
counter_penny = Counter(ZH_CATS["PENNY"])
counter_pharma = Counter(ZH_CATS["PHARMA"])
counter_property = Counter(ZH_CATS["PROPERTY"])
counter_propaganda = Counter(ZH_CATS["PROPAGANDA"])
counter_restricted_items = Counter(ZH_CATS["RESTRICTED_ITEMS"])
counter_seeds = Counter(ZH_CATS["SEEDS"])
counter_smoke = Counter(ZH_CATS["SMOKE"])
counter_surrogacy = Counter(ZH_CATS["SURROGACY"])
counter_weapon = Counter(ZH_CATS["WEAPON"])
counter_video_chat = Counter(ZH_CATS["VIDEO_CHAT"])
counter_porn = Counter(ZH_CATS["PORN"])

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Initialize the spark context
</div>

In [6]:
spark_context = SparkContext('local', 'LDA-Topic-Sandbox')

In [7]:
def counter_cosine_similarity(c1, c2):
    terms = set(c1).union(c2)
    dotprod = sum(c1.get(k, 0) * c2.get(k, 0) for k in terms)
    magA = math.sqrt(sum(c1.get(k, 0)**2 for k in terms))
    magB = math.sqrt(sum(c2.get(k, 0)**2 for k in terms))
    return dotprod / (magA * magB) * 10

<div class="lead" style="font-size:120%">
    <strong>Steps to process the corpus:</strong><br>
        1. Load each file as an individual document<br>
        2. Strip any leading or trailing whitespace<br>
        3. Convert all characters into lowercase where applicable<br>
        4. Split each document into words, separated by whitespace, semi-colons, commas, and octothorpes<br>
        5. Only keep the words that are all alphabetical characters<br>
        6. Only keep words larger than 3 characters<br>
</div>

In [8]:
directory = "/Users/bensabenteuer/Project/IdeaBox/spark-2.1.0/bin/test/*"
data = spark_context.wholeTextFiles(directory).map(lambda x: x[1])

In [9]:
tokens = data \
            .map( lambda document: document.strip().lower()) \
            .map( lambda document: re.split("[\s]", document)) \
            .map( lambda word: [x for x in word if len(x) >= 2] )

<div class="lead" style="font-size:120%">
    <strong>Steps to retrieve the vocabulary:</strong><br>
        1. Flat map the tokens -> Put all the words in one giant list instead of a list per document<br>
        2. Map each word to a tuple containing the word, and the number 1, signifying a count of 1 for that word<br>
        3. Reduce the tuples by key, i.e.: merge all the tuples together by the word, summing up the counts<br>
        4. Reverse the tuple so that the count is first...<br>
        5. ...which will allow us to sort by the word count<br>
</div>

In [10]:
termCounts = tokens                             \
    .flatMap(lambda token: token)               \
    .map(lambda word: (word, 1))                \
    .reduceByKey( lambda x,y: x + y)            \
    .map(lambda tuple: (tuple[1], tuple[0]))    \
    .sortByKey(False)

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Identify a threshold to remove the top words, in an effort to remove stop words
</div>

In [11]:
threshold_value = termCounts.take(num_of_stop_words)[num_of_stop_words - 1][0]

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Only keep words with a count less than the threshold identified above, and then index each one and collect them into a map
</div>

In [12]:
vocabulary = termCounts                         \
    .filter(lambda x : x[0] < threshold_value)  \
    .map(lambda x: x[1])                        \
    .zipWithIndex()                             \
    .collectAsMap()

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Convert the given document into a vector of word counts
</div>

In [13]:
def document_vector(document):
    id = document[1]
    counts = defaultdict(int)
    for token in document[0]:
        if token in vocabulary:
            token_id = vocabulary[token]
            counts[token_id] += 1
    counts = sorted(counts.items())
    keys = [x[0] for x in counts]
    values = [x[1] for x in counts]
    return (id, Vectors.sparse(len(vocabulary), keys, values))

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Process all of the documents into word vectors using the 'document_vector' function defined previously
</div>

In [14]:
documents = tokens.zipWithIndex().map(document_vector).map(list)

<div class="lead" style="font-size:120%">
    <strong>NOTE:</strong><br>
    Get an inverted vocabulary, so we can look up the word by it's index value
</div>

In [15]:
inv_voc = {value: key for (key, value) in vocabulary.items()}

In [16]:
with open("lda-topics.txt", 'w') as f:
    lda_model = LDA.train(documents, k=num_topics, maxIterations=max_iterations)

    topic_indices = lda_model.describeTopics(maxTermsPerTopic=num_words_per_topic)
        
    for i in range(len(topic_indices)):
        f.write("Topic #{0}\n".format(i + 1))
        for j in range(len(topic_indices[i][0])):
            topic_list.append(inv_voc[topic_indices[i][0][j]])
            f.write("{0}\t\n".format(inv_voc[topic_indices[i][0][j]].encode('utf-8')))

counter_topics = Counter(topic_list)

print "TOPICS:\n"
extracted_topics = ""

for topic in topic_list[:-1]:
    extracted_topics += topic.encode('utf-8') + ", "
extracted_topics += topic_list[-1].encode('utf-8')

print extracted_topics

TOPICS:

公司, 新年, 灿烂, 教育展, 麥當勞, 观赏, 燃放, 簿華府, 易冷, 比賽, 煙花, 爱情, 匯演, 曾经, 或许, 是否, 夜空, 國際, 一瞬间, 報電子, 寂寞, 華府, 璀璨, 瞬间, 时光, 澳門, 一生, 绚丽, 倒数, 一种, 之后, 不同, 故事, 新聞, 繁华, 投影, 公众, 观看, 产品, 过后, 刹那, 今年, 專欄, 浏阳, 永恒, 岁月, 无法, 一次, 也许, 音乐, 光阴, 烟火, 华府, 每年, 水面, 巡礼, 发射, 世界, 流星, 转瞬即逝, 流年, 现场, 凄凉, 演出, 最终, 留下, 记得, 免费, 短暂, 门票, 思念, 午夜, 知道, 结局, 人生, 安排, 放映, 最美, 浪漫, 激光, 温暖, 不愿, 网站, 生产, 不会, 火焰, 幸福, 報華府, 有人, 一点, 一地, 成为, 却是, 高空, 如今, 回忆, 不要, 夜晚, 今生, 主办单位


In [17]:
casino_score = counter_cosine_similarity(counter_topics,counter_casino)
cemetery_score = counter_cosine_similarity(counter_topics,counter_cemetery)
crowd_funding_score = counter_cosine_similarity(counter_topics,counter_crowd_funding)
cult_score = counter_cosine_similarity(counter_topics,counter_cult)
deception_score = counter_cosine_similarity(counter_topics,counter_deception)
donation_score = counter_cosine_similarity(counter_topics,counter_donation)
explosive_score = counter_cosine_similarity(counter_topics,counter_explosive)
financial_services_score = counter_cosine_similarity(counter_topics,counter_financial_services)
foreign_dating_score = counter_cosine_similarity(counter_topics,counter_foreign_dating)
forgery_score = counter_cosine_similarity(counter_topics,counter_forgery)
gender_identification_score = counter_cosine_similarity(counter_topics,counter_gender_identification)
identity_theft_score = counter_cosine_similarity(counter_topics,counter_identity_theft)
medical_services_score = counter_cosine_similarity(counter_topics,counter_medical_services)
mlm_score = counter_cosine_similarity(counter_topics,counter_mlm)
oil_score = counter_cosine_similarity(counter_topics,counter_oil)
pawn_score = counter_cosine_similarity(counter_topics,counter_pawn)
payment_services_score = counter_cosine_similarity(counter_topics,counter_pawn)
penny_score = counter_cosine_similarity(counter_topics,counter_penny)
pharma_score = counter_cosine_similarity(counter_topics,counter_pharma)
property_score = counter_cosine_similarity(counter_topics,counter_property)
propaganda_score = counter_cosine_similarity(counter_topics,counter_propaganda)
restricted_items_score = counter_cosine_similarity(counter_topics,counter_restricted_items)
seeds_score = counter_cosine_similarity(counter_topics,counter_seeds)
smoke_score = counter_cosine_similarity(counter_topics,counter_smoke)
surrogacy_score = counter_cosine_similarity(counter_topics,counter_surrogacy)
weapon_score = counter_cosine_similarity(counter_topics,counter_weapon)
video_chat_score = counter_cosine_similarity(counter_topics,counter_video_chat)
porn_score = counter_cosine_similarity(counter_topics,counter_porn)

In [19]:
print ''
print "RESULTS:\n"
print ''
print results
print ''


RESULTS:


{'SMOKE:': 0.0, 'PROPAGANDA:': 0.0, 'PAYMENT SERVICES:': 0.0, 'OIL:': 0.0, 'CEMETERY:': 0.0, 'PAWN:': 0.0, 'SURROGACY:': 0.0, 'PROPERTY:': 0.0, 'CULT:': 0.0, 'FORGERY:': 0.0, 'MEDICAL SERVICES:': 0.0, 'PENNY:': 0.0, 'WEAPON:': 0.0, 'IDENTITY THEFT:': 0.0, 'DONATION:': 0.0, 'SEEDS:': 0.0, 'GENDER IDENTIFICATION:': 0.0, 'FOREIGN DATING:': 0.0, 'RESTRICTED ITEMS:': 0.0, 'DECEPTION:': 0.0, 'MLM:': 0.0, 'CASINO:': 0.0, 'VIDEO CHAT:': 0.0, 'EXPLOSIVE:': 0.6708203932499368, 'PHARMA:': 0.0, 'CROWD_FUNDING:': 0.0, 'FINANCIAL SERVICES:': 0.0, 'PORN:': 0.0}

